<center>
<table>
  <tr>
    <td><img src="https://portal.nccs.nasa.gov/datashare/astg/training/python/logos/nasa-logo.svg" width="100"/> </td>
     <td><img src="https://portal.nccs.nasa.gov/datashare/astg/training/python/logos/ASTG_logo.png?raw=true" width="80"/> </td>
     <td> <img src="https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png" width="130"/> </td>
    </tr>
</table>
</center>

        
<center>
<h1><font color= "blue" size="+3">ASTG Python Courses</font></h1>
</center>

---

<center><h1><font color="red" size="+3">Introduction to GeoViews</font></h1></center>

## Reference Documents

* [GeoViews](https://geoviews.org/)
* [Get Started with GeoViews](https://malouche.github.io/notebooks/geoviews2.html)

_______

---

## Required Packages

```
   Matplotlib
   Numpy
   netCDF4
   Cartopy
   holoviews
   geoviews
   Xarray
```

You can install GeoViews and its dependencies using conda:

```shell
   conda install -c pyviz geoviews
```

In case the above command does not work, you may instead install a light version of GeosViews by only building the core packages:

```shell
   conda install -c pyviz geoviews-core
```

### <font color="red">Uncomment the line below if in Google Colab</font>

In [ ]:
#!apt-get install libgeos++ libproj-dev proj-data proj-bin
#!pip install cython
#!pip install cartopy
#!pip uninstall -y shapely    # cartopy and shapely aren't friends (early 2020)
#!pip install shapely --no-binary shapely
#!pip install geoviews
#!pip install xarray

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.ticker as mticker

In [ ]:
import cartopy
import cartopy.crs as crs
import cartopy.feature as cf
import cartopy.io.shapereader as shapereader
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [ ]:
import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import geoviews.tile_sources as gts
from geoviews import dim
from geoviews import opts

In [ ]:
print(f"Version of Numpy:     {np.__version__}")
print(f"Version of Pandas:    {pd.__version__}")
print(f"Version of netCDF4:   {nc4.__version__}")
print(f"Version of Xarray:    {xr.__version__}")
print(f"Version of Cartopy:   {cartopy.__version__}")
print(f"Version of HoloViews: {hv.__version__}")
print(f"Version of GeoViews:  {gv.__version__}")

# <font color="red">What is GeosViews? </font>

- `GeoViews` (Geographic visualizations for HoloViews) is built on the HoloViews library for creating flexible visualizations of multidimensional data.
- Adds a family of geographic plot types based on `Cartopy`.
- It does not create maps on its own, instead, it uses one of the `Bokeh` or `Matplotlib` as a backend to create maps. 
   - We can set whichever backend we want to use and then create maps using that backend. 
- What projections?
   - GeoViews provides a library of HoloViews Element types which make it easy to plot data on various geographic projections and other utilities to plot in geographic coordinate systems.
   - Elements are wrappers around the data along with some declaration about its dimensions.
   - Compared to HoloViews, a GeoViews element includes a Cartopy coordinate reference system (`crs` parameter) declaring the coordinate system of the data that allows the data to be automatically projected to the desired projection.
- What types of data are manipulated?
   - GeoViews is designed to work well with the Iris and Xarray libraries for working with multidimensional arrays, such as those stored in netCDF files. 
   - GeoViews also accepts data as NumPy arrays and Pandas data frames. 
   - In each case, the data can be left stored in its original, native format, wrapped in a HoloViews or GeoViews object that provides instant interactive visualizations.

>GeoViews is a Python library that makes it easy to explore and visualize geographical, meteorological, and oceanographic datasets, such as those used in weather, climate, and remote sensing research.


### <font color="blue">Setting the Backend</font>



In [ ]:
gv.extension('bokeh', 'matplotlib')

In [ ]:
%env HV_DOC_HTML=true

# <font color="red">Recall from HoloViews</font>

### <font color="blue">The Element Type</font>
- Is the HoloViews' most basic, core primitive
- Encapsulates your data to indicate how your data can be analyzed and displayed (image, curve, point plot, etc.). 
- Always preserves the raw data they are supplied.

### <font color="blue">Merging Graphs</font>
You can merge graph objects using two operations:

- `+`: Puts graphs next to each other.
- `*`: Overlays graphs on one another to create one single graph combining all individuals.

### <font color="blue">Adding Style Options</font>

We can specify:

- **Style options**: Used by the renderer (Matplotlib, Bokeh). These are things like coloring, line thickness, etc..
- **Plot options**: Control how the graphic is built. These are things like whether or not to display a title or show a grid.

We can specify these options as a dictionary and then pass them as arguments to the `opts()` method. 

# <font color="red"> Map Points</font>

- In order to plot points on the map showing locations stores, we need to use `Points()` method.
- A `Points()` object has parameters including:
   - **data**: can be a Pandas DataFrame which has latitude and longitude columns.
   - **kdims**: can be a list of two strings where the first string is column name from the DataFrame where `Longitude` information is present and the second string is where `Latitude` information is present. 
   - **vdims**: can be a list of strings as input which are column names in the DataFrame. 



Let us consider several cities around the world defined by their latitude/longitude coordinates.

In [ ]:
paris = (2.35, 48.85, "Paris")
new_york = (-73.92, 40.69, "New York")
mumbai = (72.83, 28.35, "Mumbai")
tokyo = (139.69, 35.68, "Tokyo")
moscow = (37.36, 55.45, "Moscow")
mexico_city = (-99.13, 19.43, "Mexico City")
sao_paulo = (-46.63, -23.55, "Sao Paulo")
yaounde = (11.50, 3.84, "Yaounde")
vancouver = (-123.08, 49.32, "Vacouver")
sydney = (151.20, -33.87, "Sydney")
harare = (31.0, -18.0, "Harare")

city_list = [paris, new_york, mumbai, tokyo, moscow, mexico_city,
             sao_paulo, yaounde, vancouver, sydney, harare]

#### Using Cartopy First

We can try to plot the city locations and their names with Cartopy:

In [ ]:
city_df = pd.DataFrame(
    city_list, 
    columns=['Longitude', 'Latitude', "City"]
)
city_df

In [ ]:
fig = plt.figure(figsize=(8, 5))
ax = fig.add_subplot()
ax.scatter(city_df.Longitude, city_df.Latitude);

In [ ]:
fig = plt.figure(figsize=(8, 7))
ax = fig.add_subplot(projection=crs.PlateCarree())
ax.scatter(city_df.Longitude, city_df.Latitude)
ax.coastlines()
ax.set_global()

In [ ]:
fig = plt.figure(figsize=(8, 7))
ax = fig.add_subplot(projection=crs.PlateCarree())
ax.scatter(city_df.Longitude, city_df.Latitude)
ax.stock_img()
ax.coastlines()

for i, row in city_df.iterrows():
    plt.text(row['Longitude'], row['Latitude'], row['City'],
             transform=crs.Geodetic())

#### Going back to GeoViews

We can create a GeosViews point object:

In [ ]:
cities = gv.Points(
    city_list, 
    kdims=['Longitude', 'Latitude'],
    vdims="City"
)

In [ ]:
cities.data

In [ ]:
cities.columns

In [ ]:
cities.kdims

In [ ]:
cities.vdims

We can plot the city locations:

In [ ]:
cities

The `opts()` method can be called on an instance of `Points` by giving a list of parameters and their values one after another.

In [ ]:
cities.opts(global_extent=True, 
                        width=500, 
                        height=475, 
                        size=2, color="red")

In [ ]:
cities.opts(opts.Points(global_extent=True, 
                        width=500, 
                        height=475, 
                        size=2, color="red"))

#### WMTS - Tile Sources

- GeoViews provides a number of base maps by default, provided by CartoDB, Stamen, OpenStreetMap, Esri and Wikipedia.
- They can be imported from the `geoviews.tile_sources` module.

In [ ]:
for name, ts in gts.tile_sources.items():
    print(name)

In [ ]:
gv.Layout([ts.relabel(name) for name, ts in gts.tile_sources.items()]).opts(
 "WMTS", xaxis=None, yaxis=None, width=225, height=225).cols(3)

We can select one of the maps:

In [ ]:
gv.tile_sources.OSM.opts( width=600, height=400)

We can now plot the cities on the map:

In [ ]:
gv.tile_sources.OSM * cities

In [ ]:
(gv.tile_sources.OSM * cities).opts(
    opts.Points(global_extent=True, 
    width=600, height=400, 
    size=5, color="blue")
)

#### Using Cartopy Map Projections

- GeoViews supports all projections defined in Cartopy.

In [ ]:
projections = [crs.PlateCarree, crs.RotatedPole, crs.LambertCylindrical, 
               crs.Geostationary, crs.AzimuthalEquidistant, crs.OSGB, 
               crs.EuroPP, crs.Gnomonic, crs.Mollweide, crs.OSNI, 
               crs.Miller, crs.InterruptedGoodeHomolosine, 
               crs.SouthPolarStereo, crs.Orthographic, 
               crs.NorthPolarStereo, crs.Robinson,
               crs.LambertConformal, crs.AlbersEqualArea]
proj_layout = gv.Layout([gf.coastline.relabel(group=p.__name__).opts(projection=p(), backend="matplotlib")
 for p in projections])
gv.output(proj_layout, backend="matplotlib")

In [ ]:
(gf.coastline*cities).options(opts.Points(projection=crs.PlateCarree(), 
                                          width=600, height=350, 
                                          size=3, color='red'))

#### Adding Features

In [ ]:
gf.ocean

In [ ]:
gf.ocean.options(
    'Feature', projection=crs.Geostationary(), 
    global_extent=True, height=525, width=525,
)

In [ ]:
gf.land.options(
    'Feature', projection=crs.Geostationary(), 
    global_extent=True, 
    height=525, width=525,
)

In [ ]:
(gf.ocean * gf.land * gf.coastline * gf.borders).options(
    'Feature', projection=crs.Geostationary(), 
    global_extent=True, 
    height=525, width=525,
)

In [ ]:
features = gv.Overlay([gf.ocean, gf.land, gf.rivers, 
                       gf.lakes, gf.borders, gf.coastline])

gv.output(features, backend='matplotlib', fig='svg', size=300)

In [ ]:
(features * cities).options(
    opts.Points(projection=crs.PlateCarree(), 
                width=600, height=350, size=3, color='red')
)

In [ ]:
(features * cities).options(
    opts.Points(projection=crs.Mollweide(), 
                width=800, height=400, size=3, color='red')
)

### <font color="blue"> Populations of World Cities</font>

- We want to map world cities where each dot on the map is proportional to the size of the population of that city.

In [ ]:
pop_url = "https://gist.githubusercontent.com/curran/13d30e855d48cdd6f22acdf0afe27286/raw/0635f14817ec634833bb904a47594cc2f5f9dbf8/worldcities_clean.csv"
pop_cities = pd.read_csv(pop_url)
pop_cities

Create a GeoViews Dataset using a Pandas DataFrame:

In [ ]:
population = gv.Dataset(pop_cities, kdims=['city', 'country'])

In [ ]:
type(population)

In [ ]:
population

In [ ]:
population.data

In [ ]:
population.kdims

We can create a `Points` from a `Dataset`:

In [ ]:
points = population.to(
    gv.Points, 
    ['lng', 'lat'], 
    ['population', 'city', 'country']
)

In [ ]:
type(points)

In [ ]:
points.data

In [ ]:
points.kdims

In [ ]:
points.vdims

We select the base map we want to use:

In [ ]:
#tiles = gv.tile_sources.Wikipedia
tiles = gv.tile_sources.OSM

We can now overlay the map and the points:
- Pay attention to the `hover` feature

In [ ]:
tiles * points.opts(
    color='population', 
    cmap='viridis', 
    size=dim('population')*0.000001,
    tools=['hover'], 
    global_extent=True, 
    width=600, height=600
)

### <font color="blue"> GDP Time Series of Countries</font>

- For a given year, we want to map the GDP of each country where each dot on the map is proportional to the size of the GDP of that country.

In [ ]:
url = 'https://git.mysmce.com/astg/training/py_materials/-/raw/master/geoviews/gdp_gps_years.csv'
gdp_df = pd.read_csv(url)
gdp_df

In [ ]:
gdp_df.info()

In [ ]:
gdp_dst = gv.Dataset(gdp_df, kdims=['Country', 'Year'])

In [ ]:
gdp_dst.columns

In [ ]:
gdp_points = gdp_dst.to(gv.Points, ['Longitude', 'Latitude'], 
                        ['GDP(current$)', 'Country'])

tiles = gv.tile_sources.StamenWatercolor

In [ ]:
(tiles * gdp_points).opts(
    opts.Points(size=0.000015, cmap='viridis',
                width=600, height=350, 
                tools=['hover'], 
                size_index=2, color_index=2,
                xaxis=None, yaxis=None,
                toolbar=None)
)

Saving the plot in an HTML file:

In [ ]:
renderer = gv.renderer('bokeh')
gdp_plot = tiles * gdp_points
renderer.save(gdp_plot, 'gdp_time_series_plot')

### <font color="blue">Using Natural Earth Database</font>

We can use the Natural Earth database and select the resolution of the features:

In [ ]:
graticules = cf.NaturalEarthFeature(
    category='physical',
    name='graticules_30', # graticules at 30 degree intervals
    scale='110m',
)

(gf.ocean() * gf.land() * gv.Feature(graticules, group='Lines') * gf.borders * gf.coastline).opts(
    opts.Feature('Lines', projection=crs.Robinson(), facecolor='none', edgecolor='gray'), backend='matplotlib')

We can control the scale of features by using the `scale` plot option (`10m`, `50m` and `110m`):

In [ ]:
gv.output(backend='bokeh')

(gf.ocean * gf.land.options(scale='110m', global_extent=True) * gv.Feature(graticules, group='Lines') + 
 gf.ocean * gf.land.options(scale='50m', global_extent=True) * gv.Feature(graticules, group='Lines'))

In [ ]:
(gf.ocean * gf.land.options(scale='110m', global_extent=True) * gv.Feature(graticules, group='Lines'))*cities

We can use the `geoms` method to specify a `scale` and a `bounds` to select a specific region of the globe:

In [ ]:
gf.land.geoms('50m', bounds=(-10, 40, 10, 60))

## <font color='blue'>Overlaying Gridded Data</font>

- GeoViews is designed to make full use of multidimensional gridded datasets stored in netCDF or other common formats, via the Xarray and `iris` interfaces in HoloViews.

### Basic Map

- We create synthetic geo-located data and plot them using Cartopy.

In [ ]:
nlats, nlons = 73, 145
lats = np.linspace(-np.pi / 2, np.pi / 2, nlats)
lons = np.linspace(0, 2 * np.pi, nlons)

# Create a mesh grid
lons, lats = np.meshgrid(lons, lats)
wave = 0.75 * (np.sin(2 * lats) ** 8) * np.cos(4 * lons)
mean = 0.5 * np.cos(2 * lats) * ((np.sin(2 * lats)) ** 2 + 2)

lats = np.rad2deg(lats)
lons = np.rad2deg(lons)
data = wave + mean

Do a filled countour plot:

In [ ]:
contours = gv.FilledContours((lons, lats, data), crs=crs.PlateCarree())

In [ ]:
contours.data

In [ ]:
contours

We can now include coaslines:

In [ ]:
(contours*gf.coastline).opts(opts.FilledContours(
    levels=8, color_levels=10, 
    cmap='RdBu',
    colorbar=True, 
    width=600, height=300, 
    projection=crs.PlateCarree())
                            )

### Manipulating Gridded Time Series Dataset

We use here GEOS-5 MERRA2 datasets.

#### We first read the netCDF file using Xarray

In [ ]:
import urllib
url = "https://git.mysmce.com/astg/training/py_materials/-/raw/master/geoviews/MEERA2_ps_monthly_2020.nc4"
loc_fname = "MERRA2_ps_monthly_2020.nc4"
urllib.request.urlretrieve(url, loc_fname)

In [ ]:
%env HDF5_USE_FILE_LOCKING=FALSE

In [ ]:
xr.set_options(file_cache_maxsize=10)

In [ ]:
ps_xr = xr.open_dataset(loc_fname)
ps_xr

#### We wrap the Xarray Dataset into a GeoViews Dataset Element

We need to define the:
- Key dimensions: `kdims`
- Value dimensions: `vdims`

In [ ]:
kdims = ['time', 'lon', 'lat']
vdims = ['ps']

gv_dataset = gv.Dataset(ps_xr, kdims=kdims, vdims=vdims)

In [ ]:
print(repr(gv_dataset))

Create a custom value formatter for the time dimension so that it is readable by humans:

In [ ]:
gv_dataset.get_dimension_type('time')

In [ ]:
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

Plot the data as an image:

In [ ]:
gv_dataset.to(gv.Image, ['lon', 'lat'])

In [ ]:
(gv_dataset.to.image(['lon', 'lat']) * gf.coastline).opts(
    opts.Image(cmap='jet', xaxis=None, yaxis=None))

In [ ]:
ps_curve = hv.Curve(gv_dataset.select(lon=0, lat=10), kdims=['time'])

ps_map = gv_dataset.to(gv.Image,['lon', 'lat']) * gv.Points([(0,10)])

(ps_map + ps_curve).opts(
    opts.Curve(aspect=2, xticks=4, xrotation=15),
    opts.Points(color='k', global_extent=True)
)

Do contour plots:

In [ ]:
ps_contour = gv_dataset.to(gv.FilledContours, ['lon', 'lat'])
ps_contour

In [ ]:
ps_contour.opts(width=600, height=300)

In [ ]:
(ps_contour * gf.coastline).opts(width=600, height=300)

In [ ]:
(ps_contour*gf.coastline).opts(
    opts.FilledContours(levels=8, color_levels=10, 
                        cmap='jet', colorbar=True, 
                        width=600, height=300, 
                        projection=crs.PlateCarree())
)

### Using `hvplot`

In [ ]:
import hvplot.xarray

In [ ]:
ps_xr.hvplot(
    groupby="time",
    cmap='jet',
)

In [ ]:
ps_xr.hvplot(
    groupby="time", 
    cmap='jet', 
    widget_type="scrubber",
    widget_location="bottom",
)

In [ ]:
ps_xr.hvplot.contourf(
    groupby="time", 
    cmap='jet', 
    levels=10,
    widget_type="scrubber",
    widget_location="bottom",
)